# Обработка пропушенных значений

In [1]:
import pandas as pd
import numpy as np

RANDOM_STATE = 42

Пропуски в этом датасете обозначены как `?`. **Проверка количества пропусков в каждом столбце**

In [6]:
for c in df.columns:
    print(c, len(df[df[c] == '?']))

symboling 0
normalized-losses 41
make 0
fuel-type 0
aspiration 0
num-of-doors 2
body-style 0
drive-wheels 0
engine-location 0
wheel-base 0
length 0
width 0
height 0
curb-weight 0
engine-type 0
num-of-cylinders 0
engine-size 0
fuel-system 0
bore 4
stroke 4
compression-ratio 0
horsepower 2
peak-rpm 2
city-mpg 0
highway-mpg 0
price 4


С символьным обозначением проусков работать неудобно, т.к. не применишь стандартные функции. 

**Заменим все '?' на  None**

In [7]:
df.replace('?', value=np.nan, inplace=True)

**Для категориального признака земеняем пропуски на самое популярное значение этого признака**

In [10]:
# определяем самое популярное значение
df['num-of-doors'].value_counts()

num-of-doors
four    113
two      86
Name: count, dtype: int64

In [11]:
# заменяем
df['num-of-doors'].fillna('four', inplace=True) 

**Для числовых признаков заменяем пропуски на средние значения этих признаков**

In [12]:
# хоть признаки и числовые, но у них тип object. Меняем тип на float
df[['bore', 'stroke', 'horsepower', 'peak-rpm']] = df[['bore', 'stroke', 'horsepower', 'peak-rpm']].astype('float64', copy=False)

In [13]:
# считаем средние
m_bore = df.bore.mean()
m_stroke = df.stroke.mean()
m_horsepower = df.horsepower.mean()
m_peak_rpm = df['peak-rpm'].mean()
print(m_peak_rpm)

5117.587939698493


In [14]:
# заполняем пропуски средними значениями
df.bore.fillna(m_bore, inplace=True)
df.stroke.fillna(m_stroke, inplace=True)
df.horsepower.fillna(m_horsepower, inplace=True)
df['peak-rpm'].fillna(m_peak_rpm, inplace=True)

**Проверяем, что пропуски заполнились**

In [15]:
df.isna().sum()

symboling             0
normalized-losses    37
make                  0
fuel-type             0
aspiration            0
num-of-doors          0
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  0
stroke                0
compression-ratio     0
horsepower            0
peak-rpm              0
city-mpg              0
highway-mpg           0
price                 0
dtype: int64

**Пропуски в столбце `normalized-losses` предскажем при помощи линейной регрессии** по признакам
`symboling`, `wheel-base`, `length`, `width`, `height`, `curb-weight`, `engine-size`, `compression-ratio`, `city-mpg`, `highway-mpg` и заполним их предсказаниями

In [16]:
from sklearn.linear_model import LinearRegression

# выделяем индексы, где известно значение 'normalized-losses' (train выборка) и неизсестно (test выборка)
ind_train = df[df['normalized-losses'].notna()].index
ind_test = df[df['normalized-losses'].isna()].index

# формируем тренировочную выборку
X = df.loc[ind_train, ['symboling', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-size', 'compression-ratio', 'city-mpg', 'highway-mpg']]
y = df.loc[ind_train, ['normalized-losses']]

In [17]:
# строим модель
fill_missed_model = LinearRegression()
fill_missed_model.fit(X, y)

LinearRegression()

In [18]:
# находим предсказание
X_test = df.loc[ind_test, ['symboling', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-size', 'compression-ratio', 'city-mpg', 'highway-mpg']]
pred = fill_missed_model.predict(X_test)
pred[0]

array([168.07249262])

In [ ]:
# заполняем датасет предстказаниями. Учитываем, что в df индексация общая по датасету, а в predict - своя от нуля до количества пропущенных значений
count = 0
for i in ind_test:
    df.loc[i, ['normalized-losses']] = pred[count]
    count += 1